# Download a batch of FMRC Files
Once a HycomDataset entry has been defined from the Hycom catalog, the updateFMRCData method takes care of identifying avaiable FMRCs as well as staging and downloading the Data files. Below is a demonstration using default options except for limit which is set to 12 to avoid creating an uncessasary server load.

Expore Options For Download Jobs
A series of types have been created to specify options:
FMRCSubsetOptions - Hycom related options for the Thredds server
FMRCDownloadOptions - Options realted to how to request files from the server
FMRCDownloadJobOptions - C3 Batch Job Options for managing the downloads
The next cell displays the default settings for each option type.

In [1]:
# Default subset options
print(c3.FMRCSubsetOptions())
# Default download options
print(c3.FMRCDownloadOptions())
# Use default download options
print(c3.FMRCDownloadJobOptions())

c3.FMRCSubsetOptions(
 timeStride=1,
 vars='surf_el,salinity,water_temp,water_u,water_v',
 disableLLSubset='on',
 disableProjSubset='on',
 horizStride=1,
 vertStride=1,
 addLatLon='false',
 accept='netcdf4')
c3.FMRCDownloadOptions(
 externalDir='hycom-data',
 maxTimesPerFile=1,
 maxForecastDepth=-1,
 defaultHycomDatasetId='GOMu0.04_901m000_FMRC_1.0.1')
c3.FMRCDownloadJobOptions(batchSize=4, limit=-1)


In [20]:
# Ensure we have a Dataset entry for the desired catalog
cat_url = "https://tds.hycom.org/thredds/catalog/GOMu0.04/expt_90.1m000/FMRC/runs/catalog.xml"
gom_dataset = c3.HycomDataset.upsertHycomDatasetFromCatalog(url = cat_url)

# Create an updateFMRCData job.
# Use defaults for Subset ond download options
# Limit # of files downloaded for demo

data_dir = 'hycom-data'
job = c3.HycomFMRC.updateFMRCData(
    fmrcSubsetOptions = c3.FMRCSubsetOptions(),
    fmrcDownloadOptions = c3.FMRCDownloadOptions(
        **{
            'externalDir': data_dir
        }
    ),
    fmrcDownloadJobOptions = c3.FMRCDownloadJobOptions()
)
job.status()

c3.BatchJobStatus(
 started=datetime.datetime(2021, 10, 7, 11, 32, 20, tzinfo=datetime.timezone.utc),
 startedby='dadams@illinois.edu',
 status='running')

In [21]:
# Monitor the job status until completed and display total # of files downloaded
import time
from IPython.display import clear_output
status = job.status()
while status.status == 'running':
    clear_output()
    status = job.status()
    filecount = c3.FMRCFile.fetchCount(spec={'filter':"status=='downloaded'"})
    gom_dataset = c3.HycomDataset.fetch().objs[0]
    print(f"Download count: {filecount}")
    print(f"FMRC Archive Size: {round(gom_dataset.fmrcArchiveSize/(1024**3),2)} GiB")
    print (status)
    time.sleep(10)

Download count: 1861
FMRC Archive Size: 28.0 GiB
c3.BatchJobStatus(
 started=datetime.datetime(2021, 10, 7, 11, 32, 20, tzinfo=datetime.timezone.utc),
 startedby='dadams@illinois.edu',
 completed=datetime.datetime(2021, 10, 7, 12, 2, 12, tzinfo=datetime.timezone.utc),
 status='completed',
 newBatchSubmitted=False)


In [ ]:
job.cancel()

In [11]:
job.status()

c3.BatchJobStatus(
 started=datetime.datetime(2021, 10, 4, 9, 50, 33, tzinfo=datetime.timezone.utc),
 startedby='dadams@illinois.edu',
 status='running',
 newBatchSubmitted=True)

In [ ]:
gom_dataset = c3.HycomDataset.fetch().objs[0]
print(f"FMRC Archive Size: {round(gom_dataset.fmrcArchiveSize/(1024**3),2)} GiB")

In [ ]:
# Check on the records
for fmrc in c3.HycomFMRC.fetch(spec={'include': "this,fmrcDataArchive", 'filter': "expired==true"}).objs:
    print (fmrc.fmrcDataArchive)
    #filecount = c3.FMRCFile.fetchCount(spec={'filter':"dataArchive=="})
    #print(filecount)

In [ ]:
c3.FMRCFile.fetchCount(spec={'filter':"dataArchive=='GOMu0.04/expt_90.1m000/FMRC/runs/GOMu0.04_901m000_FMRC_RUN_2021-09-14T12:00:00Z'"})

In [ ]:
# Pick the first avalable file, open it and print the NetCDF metadata
file = c3.FMRCFile.fetch().objs[1]
ds = c3.HycomUtil.nc_open(file.file.url,'/tmp')
print(ds)
c3.HycomUtil.nc_close(ds, file.file.url, '/tmp')

In [ ]:
# # Cleanup - uncomment to remove all FMRC File records
# print(f"Removed {c3.FMRCFile.removeAll()} FMRCFile records.")
# print(f"Removed {c3.FMRCDataArchive.removeAll()} FMRCDataArchive records.")
# print(f"Removed {c3.HycomFMRC.removeAll()} HycomFMRC records.")
# print(f"Removed {c3.HycomDataset.removeAll()} HycomDataset records")
# files = c3.FileSystem.inst().listFiles(data_dir)
# if files.files:
#     print(f"Deleting {len(files.files)} files")
#     c3.FileSystem.inst().deleteFilesBatch(files.files)
# print("Done.")